# Authors first and last commit date

This notebooks focus on retrieving data from ES and using Pandas to perform some basic analysis on it.


## Import libraries

First we need to import those Python modules we are going to use. We could import them at any point before using them.

In [50]:
import certifi
import configparser
import json
import os
import sys

from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search

## Declaring functions
We can define new functions at any point. In this case we decided to declare 'create_conn' function here at the begining because is a generic function. In fact we could create a separate module with this kind of general functions and import that module in every notebook we need to create an ES connection.

Notice we are pointing to '../.settings' file to use same config as in plain script version of this code.

In [51]:
def create_conn():
    """Creates an ES connection from ''.settings' file.

    ''.settings' contents sample:
    [ElasticSearch]

    user=john_smith
    password=aDifficultOne
    host=my.es.host
    port=80
    path=es_path_if_any
    """

    parser = configparser.ConfigParser()
    parser.read('../.settings')

    section = parser['ElasticSearch']
    user = section['user']
    password = section['password']
    host = section['host']
    port = section['port']
    path = section['path']

    connection = "https://" + user + ":" + password + "@" + host + ":" + port \
                + "/" + path

    es_read = Elasticsearch([connection], use_ssl=True,
                            verity_certs=True, ca_cert=certifi.where(),
                            scroll='300m', timeout=1000)

    return es_read

## Querying ES
Following code creates an ES connection and executes the query to get, for each author, first and last commit dates. 

In order to get unique authors, a bucket is created using 'author_uuid' field.

Notice that a filter is applied to get data for whole years. Thus, we exclude data for current year.

In [52]:
"""Query ES to get first and last commit of each author together with
some extra info like .
"""
es_conn = create_conn()

# Create search object
s = Search(using=es_conn, index='git')

# FILTER: retrieve commits before given year
s = s.filter('range', grimoire_creation_date={'lt': 'now/y'})

# Bucketize by uuid and get first and last commit (commit date is stored in
# author_date field)
s.aggs.bucket('authors', 'terms', field='author_uuid', size=10000000) \
    .metric('first', 'top_hits',
            _source=['author_date', 'author_org_name', 'author_uuid', 'project'],
            size=1,
            sort=[{"author_date": {"order": "asc"}}]) \
    .metric('last_commit', 'max', field='author_date')

# Sort by commit date
s = s.sort("author_date")

result = s.execute()

# Print results

From here, we can start playing with the data, but first we can print those results to have a look at them.

Notice we can use variables from other cells that were executed previosly (look at numbers between square brackets if not sure about execution order).

In [53]:
from pprint import pprint

result_buckets = result.to_dict()['aggregations']['authors']['buckets']

pprint(result_buckets)

[{'doc_count': 29838,
  'first': {'hits': {'hits': [{'_id': '40a74d03976c4810bf4b5c57b94c9b79c9b6b087',
                               '_index': 'git_bitergia_171229_enriched_171229',
                               '_score': None,
                               '_source': {'author_date': '2012-05-19T07:41:30',
                                           'author_org_name': 'Bitergia',
                                           'author_uuid': '57de6040b8a98143a5eb938109e4b0a907038d77',
                                           'project': 'Bitergia'},
                               '_type': 'items',
                               'sort': [1337413290000]}],
                     'max_score': None,
                     'total': 29838}},
  'key': '57de6040b8a98143a5eb938109e4b0a907038d77',
  'last_commit': {'value': 1514475862000.0,
                  'value_as_string': '2017-12-28T15:44:22.000Z'}},
 {'doc_count': 21749,
  'first': {'hits': {'hits': [{'_id': '86ffac32683d0b3c2234da65c9f43b44e8

## Create Pandas dataframe
In order to work with data, we create a Pandas dataframe where each row will contain:
* Author UUID
* First commit date
* Last commit date
* Author org name
* Project

In [54]:
import pandas as pd
from datetime import datetime

# Get a dataframe with each author and their first commit
buckets = []
for bucket_author in result_buckets:
    author = bucket_author['key']

    first = bucket_author['first']['hits']['hits'][0]
    first_commit = first['sort'][0]/1000
    last_commit = bucket_author['last_commit']['value']/1000
    org_name = first['_source']['author_org_name']
    project = first['_source']['project']
    
    buckets.append({
            'first_commit': datetime.utcfromtimestamp(first_commit),
            'last_commit': datetime.utcfromtimestamp(last_commit),
            'author': author,
            'org': org_name,
            'project': project
    })
    
authors_df = pd.DataFrame.from_records(buckets)
authors_df.sort_values(by='first_commit', ascending=False,
                        inplace=True)

pprint(authors_df)

                                      author        first_commit  \
82  d3ec9b39d81b178650488056026b3bfaad520294 2017-12-07 18:54:53   
74  74a144a3e005ade6e772355921710f206fd0e3b1 2017-10-17 22:46:36   
61  3e07ffee1f3dc6eac7be34d46b0423ab81e2eac7 2017-10-10 16:27:29   
68  12ece20936dd5c1625548c0bc543da6314785e51 2017-10-03 08:58:14   
44  0c0660cc3c4aa581bf8349cdff17b99d24bfd4bb 2017-09-21 12:01:50   
18  3af32d529fce2e7dc632bb26fea5ed4abcb3fcd6 2017-09-07 14:46:30   
62  60f91e9419cbadd93686b6db9fdde3ba797c7eeb 2017-09-01 13:05:18   
67  077ecab008c9bcedb62d88f228f006f0be74155d 2017-08-04 12:03:03   
80  c6e8d9e2b5c7cc2f174df3d20487a6a5a08824ae 2017-06-29 12:45:26   
81  c7dd3e3fc036268ed07d5a570655ba02ad4f2035 2017-06-07 11:12:08   
73  6ca63954e4ee2df6be35b39384720e4d4c0908d6 2017-04-10 16:46:28   
58  00d388d70234b2e2260b874ba121498a23b5b968 2017-04-07 14:47:12   
66  05f04576638790d55b20d451b7cd10c0ee257731 2017-04-03 22:45:24   
56  9cc2fc42cd9337dccb3a6f6b305649d9fa13881a 201

## Newcomers per year
Next we will use pandas to group data and count the number of newcomers per year and organization.

In [55]:
# Group by year of first commit and project, counting number of authors
first_df = authors_df.groupby([authors_df.first_commit.dt.year, authors_df.org])\
                        .agg({'author': pd.Series.nunique})
first_df = first_df.reset_index()
first_df.rename(columns={"first_commit": "year", "author": "newcomers"}, inplace=True)
first_df = first_df.sort_values(by=['year', 'newcomers'], ascending=[False, False])

In [56]:
# Get top 20 projects based on newcomers from 2008
newcomers_df = pd.DataFrame()
for year in first_df['year'].unique():
    if year > 2008:
        year_df = first_df.loc[first_df['year'] == year].head(20)
        newcomers_df = pd.concat([newcomers_df, year_df])
        
pprint(newcomers_df)

    year                          org  newcomers
20  2017                      Unknown         13
19  2017                     Bitergia          4
17  2016                     Bitergia          3
18  2016                      Unknown          3
16  2015                      Unknown         10
15  2015                     Bitergia          1
14  2014                      Unknown          4
11  2014                     Bitergia          2
12  2014           Eclipse Foundation          1
13  2014  Universidad Rey Juan Carlos          1
9   2013                     Bitergia          5
10  2013         Wikimedia Foundation          1
8   2012                      Unknown         12
6   2012                     Bitergia          2
7   2012                Geeknet, Inc.          1
4   2011                Geeknet, Inc.          5
5   2011                      Unknown          2
2   2010                Geeknet, Inc.          6
3   2010                      Unknown          1
0   2009            

## People Leaving


In [57]:
# Group by year of last commit and project, counting number of authors
last_df = authors_df.groupby([authors_df.last_commit.dt.year, authors_df.org])\
                        .agg({'author': pd.Series.nunique})
last_df = last_df.reset_index()
last_df.rename(columns={"last_commit": "year", "author": "leaving"}, inplace=True)
last_df = last_df.sort_values(by=['year', 'leaving'], ascending=[False, False])

In [58]:
# Get top 20 projects based on newcomers from 2008
leaving_df = pd.DataFrame()
for year in last_df['year'].unique():
    if year > 2008:
        year_df = last_df.loc[last_df['year'] == year].head(20)
        leaving_df = pd.concat([leaving_df, year_df])
        
pprint(leaving_df)

    year                          org  leaving
14  2017                     Bitergia       15
16  2017                      Unknown       15
15  2017  Universidad Rey Juan Carlos        1
13  2016                      Unknown        5
12  2016                     Bitergia        1
10  2015                      Unknown        9
11  2015         Wikimedia Foundation        1
8   2014           Eclipse Foundation        1
9   2014                      Unknown        1
7   2013                     Bitergia        1
6   2012                      Unknown       12
5   2012                Geeknet, Inc.        7
3   2011                Geeknet, Inc.        3
4   2011                      Unknown        3
1   2010                Geeknet, Inc.        6
2   2010                      Unknown        1
0   2009                      Unknown        1


## Merge both dataframes

Now we put everything together to keep track of people joining and leaving the community through years.

In [59]:
final_df = newcomers_df.merge(leaving_df, on=['year','org'], how='outer')
final_df = final_df.fillna(0)
final_df = final_df.sort_values(by=['year', 'org'], ascending=[False, False])

pprint(final_df)

    year                          org  newcomers  leaving
0   2017                      Unknown       13.0     15.0
21  2017  Universidad Rey Juan Carlos        0.0      1.0
1   2017                     Bitergia        4.0     15.0
3   2016                      Unknown        3.0      5.0
2   2016                     Bitergia        3.0      1.0
22  2015         Wikimedia Foundation        0.0      1.0
4   2015                      Unknown       10.0      9.0
5   2015                     Bitergia        1.0      0.0
6   2014                      Unknown        4.0      1.0
9   2014  Universidad Rey Juan Carlos        1.0      0.0
8   2014           Eclipse Foundation        1.0      1.0
7   2014                     Bitergia        2.0      0.0
11  2013         Wikimedia Foundation        1.0      0.0
10  2013                     Bitergia        5.0      1.0
12  2012                      Unknown       12.0     12.0
14  2012                Geeknet, Inc.        1.0      7.0
13  2012      